In [31]:
import math

def myarctan(x, n):
    z = x
    # We use the value of z and iteratively compute z/(1 + root(1 + z^2)) n times
    for i in range(n):
        z = z/(1+math.sqrt(1+(z**2)))
    # Here we set l to z after the argument has been reduced n times where l represents the first term in the taylor series for arctan
    # The taylor series is arctan(x) = x - x^3/3 + x^5/5 - ... so we take arctan(z) = z assuming z is small
    l = z
    # Now we have to scale our reduction back from arctan(z) so that we get the approximate value of arctan(x)
    # We multiplyt l by 2 n times since we reduced x n times
    for i in range(n):
        l = l*2
    return l
    
def cordic_tan(angle, n):
    # Normalize the input angle to the range [0, π/2]
    # if the angle is greater than pi/2 we keep subtracting pi
    while angle > math.pi / 2:
        angle -= math.pi
    # once below zero we add pi to get in the right interval
    while angle < 0:
        angle += math.pi

    # Initialize x0 = 1 and y0 = 0 in the algorithm that will be shifted 
    x = 1.0
    y = 0.0
    
    # Iterate n times to calculate the values of k for the different arctan values for arctan(1) to arctan (10^-n)
    for i in range(n):
        # Initialize k for the current arctan value
        k = 0
        # This will be the argument of the arctan
        power_ten = 10**(-i)
        # Calculate the value of the arctan using previous myarctan function
        cur_arctan = myarctan(power_ten, 50)
        
        # Loop until the value of the angle minus the arctan is below zero and calculate the value of ki 
        # i.e the number of times the arctan value fits in the current value of t
        while angle - cur_arctan > 0:
            angle = angle - cur_arctan
            k = k + 1

        # Here we perform the shifting of x and y vectors based on the found value of k 
        # and the current power of 10 as the argument to arctan
        for j in range(k):
            x = x - power_ten * y
            y = y + power_ten * x

    # Once we can't make the input angle any smaller, we return the value of tan which is the shifted y/x
    return y/x

# This demonstrates the calculations of tan by computing tan from 0.1 to 1.5 and showing the errors
print("TAN")
for i in range(1, 16):
    print("")
    x = i/10
    n = 15
    mtan = cordic_tan(x, n)
    print("x: " + str(x) + " n: " + str(n) + " cordic_tan(x): " + str(mtan) + " absolute error: " + str(abs(mtan - math.tan(x))) + " relative error: " + str(abs((mtan - math.tan(x))/math.tan(x))))

# This computes the sin of given angle using the cordic_tan implementation
def sin_from_tan(angle, n):
    # First we need to scale the given angle to be between 0 and 2pi
    while angle > 2*math.pi:
        angle -= 2 * math.pi

    # Then we compute tan using the cordic tan algorithm
    tan = cordic_tan(angle, n)

    # Compute sin using the formula for sin in terms of tan sin x = +- tan x / square_root(1 + tan^2x)
    sin = tan / math.sqrt(1 + tan ** 2)

    # Finally, we account for the fact that the sign of tan and sin is different in the second and third quadrant
    # so we take negative in the formula
    if math.pi/2 <= angle <= 3 * math.pi / 2:
        return -1 * sin
    #Otherwise we take positive in the formula
    else:
        return sin

# This computes the cos of given angle using the cordic_tan implementation
def cos_from_tan(angle, n):
    # First we need to scale the given angle to be between 0 and 2pi
    while angle > 2*math.pi:
        angle -= 2 * math.pi

    # Then we compute tan using the cordic tan algorithm
    tan = cordic_tan(angle, n)

    # Compute sin using the formula for cos in terms of tan cos x = +- 1 / square_root(1 + tan^2x)
    cos = 1 / math.sqrt(1 + tan ** 2)

    # Finally, we account for the fact that the sign of tan and cos is different in the third and fourth quadrant
    # so we take negative in the formula
    if math.pi <= angle <= 2 * math.pi:
        return -1 * cos
    #Otherwise we take positive in the formula
    else:
        return cos

# This demonstrates the calculations of sin by computing sin from 0.1 to 1.5 and showing the errors
print("SIN")
for i in range(1, 16):
    print("")
    x = i/10
    n = 15
    msin = sin_from_tan(x, n)
    print("x: " + str(x) + " n: " + str(n) + " sin_from_tan(x): " + str(msin) + " absolute error: " + str(abs(msin - math.sin(x))) + " relative error: " + str(abs((msin - math.sin(x))/math.sin(x))))


# This demonstrates the calculations of cos by computing cos from 0.1 to 1.5 and showing the errors
print("COS")
for i in range(1, 16):
    print("")
    x = i/10
    n = 15
    mcos = cos_from_tan(x, n)
    print("x: " + str(x) + " n: " + str(n) + " cos_from_tan(x): " + str(mcos) + " absolute error: " + str(abs(mcos - math.cos(x))) + " relative error: " + str(abs((mcos - math.cos(x))/math.cos(x))))

    

TAN

x: 0.1 n: 15 cordic_tan(x): 0.10033466905210495 absolute error: 3.0333456024767003e-09 relative error: 3.023227703274234e-08

x: 0.2 n: 15 cordic_tan(x): 0.20169965201734844 absolute error: 0.0010103834913240406 relative error: 0.004984378246437699

x: 0.3 n: 15 cordic_tan(x): 0.30622038793940565 absolute error: 0.0031158616702175457 relative error: 0.01007273371339346

x: 0.4 n: 15 cordic_tan(x): 0.416290970713063 absolute error: 0.0065022480250988 relative error: 0.015379262809618712

x: 0.5 n: 15 cordic_tan(x): 0.5348102646322879 absolute error: 0.011492225211502616 relative error: 0.021036377144809826

x: 0.6 n: 15 cordic_tan(x): 0.6655177933684316 absolute error: 0.018619014973260684 relative error: 0.02721533872500164

x: 0.7 n: 15 cordic_tan(x): 0.8135265309303252 absolute error: 0.02876184953275407 relative error: 0.03414727093461882

x: 0.8 n: 15 cordic_tan(x): 1.0295324132495838 absolute error: 0.00010614380078011187 relative error: 0.00010308840908617987

x: 0.9 n: 15 c

In [32]:
import math

def myarctan(x, n):
    z = x
    # We use the value of z and iteratively compute z/(1 + root(1 + z^2)) n times
    for i in range(n):
        z = z/(1+math.sqrt(1+(z**2)))
    # Here we set l to z after the argument has been reduced n times where l represents the first term in the taylor series for arctan
    # The taylor series is arctan(x) = x - x^3/3 + x^5/5 - ... so we take arctan(z) = z assuming z is small
    l = z
    # Now we have to scale our reduction back from arctan(z) so that we get the approximate value of arctan(x)
    # We multiplyt l by 2 n times since we reduced x n times
    for i in range(n):
        l = l*2
    return l

# This function computes arctan of A using the cordic shifting and adding algorithm
def cordic_arctan (A, n):
    # First we save the sign of the input in order to multiply the result by it at the end
    sign = 1
    if A < 0:
        sign = -1

    # We initialize z and y which will slowly be shifted closer and closer to the angle
    x = 1
    y = abs(A)

    # We initialize the angle to 0 the values of smaller angles will be slowly added to it
    angle = 0
    for i in range(n):
        # Initialize k for the current arctan value
        k = 0
        
        # This will be the argument of the arctan
        power_ten = 10**(-i)
        
        # Calculate the value of the arctan using previous myarctan function
        cur_arctan = myarctan(power_ten, 15)

        # We shift the values of x and y backward based on current power of 10 until right before y becomes negative
        # The power of 10 represent a tan of small angles starting with theta = 1
        while y - x * power_ten > 0:
            y = y - x * power_ten
            x = x + y * power_ten

            # we need to keep track of how many times the given power of 10 shifter
            k = k + 1

        # Finally for each iteration we add k times the arcatan of the power of 10 where each time
        # the arctan gets smaller until we get as close as possible to actual angle
        angle = angle + k * cur_arctan

    # here we incorperate the sign and add the residual angle to computed angle which we assume for small r, sin r = r
    return sign * (angle + y/x)

# This demonstrates the calculations of arctan by computing arctan from -0.9 to 0.9 and showing the errors
print("ARCTAN")
for i in range(-9, 10):
    print()
    a = i/10
    n = 15
    marctan = cordic_arctan(a, n)
    if math.atan(a) != 0:
        print("x: " + str(a) + " n: " + str(n) + " cordic_arctan(x): " + str(marctan) + " absolute error: " + str(abs(marctan - math.atan(a))) + " relative error: " + str(abs((marctan - math.atan(a))/math.atan(a))))
    else:
        print("x: " + str(a) + " n: " + str(n) + " cordic_arctan(x): " + str(marctan) + " absolute error: " + str(abs(marctan - math.atan(a))))
   
# This function computes the arcsin using cordic arctan
# To compute the arcsin we use the formula arcsin A = arctan (A / sqrt(1 - A^2)
def arcsin_from_arctan (A, n):
    # First we calculate the input to arctan
    input_to_arctan = A / math.sqrt(1 - A**2)
    # Then return the arctan of the computed value using the cordic arctan function
    return cordic_arctan(A, n)

print()
# This demonstrates the calculations of arcsin by computing arcsin from -0.9 to 0.9 and showing the errors
print("ARCSIN")
for i in range(-9, 10):
    print("")
    a = i/10
    n = 15
    marcsin = arcsin_from_arctan(a, n)
    if math.asin(a) != 0:
        print("x: " + str(a) + " n: " + str(n) + " arcsin_from_arctan(x): " + str(marcsin) + " absolute error: " + str(abs(marcsin - math.asin(a))) + " relative error: " + str(abs((marcsin - math.asin(a))/math.asin(a))))
    else: 
        print("x: " + str(a) + " n: " + str(n) + " arcsin_from_arctan(x): " + str(marcsin) + " absolute error: " + str(abs(marcsin - math.asin(a))))
    

ARCTAN

x: -0.9 n: 15 cordic_arctan(x): -0.7524518960822133 absolute error: 0.01963679429570664 relative error: 0.026796383218406283

x: -0.8 n: 15 cordic_arctan(x): -0.6916020713165799 absolute error: 0.016861129093027172 relative error: 0.024989041034715814

x: -0.7 n: 15 cordic_arctan(x): -0.6247298294356456 absolute error: 0.014003865046437025 relative error: 0.02292986685189059

x: -0.6 n: 15 cordic_arctan(x): -0.5514641877975588 absolute error: 0.011044687526974628 relative error: 0.020437248325503858

x: -0.5 n: 15 cordic_arctan(x): -0.47169852357760883 absolute error: 0.008050914576802681 relative error: 0.017364296548736613

x: -0.4 n: 15 cordic_arctan(x): -0.38573211616014297 absolute error: 0.005225739047778066 relative error: 0.013733643802334716

x: -0.3 n: 15 cordic_arctan(x): -0.2942559818389795 absolute error: 0.0027991873611123985 relative error: 0.009604124570597257

x: -0.2 n: 15 cordic_arctan(x): -0.19840063647312675 absolute error: 0.0010050766232459762 relative er